In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install locationtagger

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 77.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 96.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=a78b23e4be5c34030bef2dbe

### Import Packages

In [ ]:
import requests
import numpy as np
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os
import ssl
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from urllib import request
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from shapely.geometry import Point, Polygon
from requests.exceptions import ConnectTimeout
import spacy
import nltk
import locationtagger
# import geojson
# import geopandas as gpd
import matplotlib.pyplot as plt

## 1. Text Extraction
### 1.1 Web Crawling with URL

In [ ]:
max_page_num =1000
news_url = []
contents = []
titles = []
times = []
regex = r"^.*[\n:].*$"

for i in range(max_page_num+1):
  url = ('https://newslookup.com/results?p='+str(i)+'&q=earthquake&dp=5&mt=-1&ps=10&s=&cat=gi100&fmt=&groupby=no&site=&dp=5&tp=Y2020')
  resp = requests.get(url,verify=False)
  html = BeautifulSoup(resp.content,'html.parser')
  articles = html.select('div#results > a.title ')
  time = html.select('div#results > span.stime')
   
  for j in articles:
    titles.append(j.get_text())
    news_url.append(j.attrs['href'])

  for k in time:
    times.append(k.get_text())

In [ ]:
df1 = pd.DataFrame(titles,columns=['Title'])
df1.insert(1,'Time',times,True)
df1.insert(2,'Url',news_url,True)
df1.to_csv('news1.csv',index=False)

### 1.2 Body Text Extraction

In [ ]:
## Contents crawling ## 
for a in news_url:
    try:
        main_url = a
        original_html = requests.get(main_url, timeout= 10)
        soup = BeautifulSoup(original_html.text,'html.parser')
        #print(soup)
        data = soup.find_all('p')
        #print(data)
        data1 = []
        for i in data[:]:
            data1.append(i.get_text())
            #^((?!Please enter a search item).)*$
        for n in range(len(data1)):
            match = re.search(regex, data1[n])
            if match:
                data1[n] = ""
        data2 = ' '.join(data1)
        #print("===================\n", data2)     
        contents.append(data2)
    except ConnectTimeout:
        contents.append(' ')
    except requests.exceptions.Timeout:
        contents.append(' ')
    except requests.exceptions.SSLError:
        contents.append(' ')
    except requests.exceptions.ConnectionError:
        contents.append(' ')

df = pd.DataFrame(titles,columns=['Title'])
df.insert(1,'Time',times,True)
df.insert(2,'Url',news_url,True)
df.insert(3,'Body',contents,True)
df.to_csv('news.csv',index=False)
len(df)

In [ ]:
# sp_filter = (df['Time'] > start_date) & (df['Time'] <= end_date)
# sp = df.loc[sp_filter]

# su_filter = (df['Time'] > start_date) & (df['Time'] <= end_date)
# su = df.loc[su_filter]

# fa_filter = (df['Time'] > start_date) & (df['Time'] <= end_date)
# fa = df.loc[fa_filter]

# sp_filter = (df['Time'] > start_date) & (df['Time'] <= end_date)
# wn = df.loc[wn_filter]

## 2. Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gdrive_path = "/content/drive/Shared drives/NLP study/Seonho Project/"

In [ ]:
#pip install summa
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv(gdrive_path + 'news_whole.csv').drop_duplicates(subset = ['Title']).dropna().reset_index()

### 2.1 Text Summarization

In [ ]:
pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54408 sha256=a8c77f9d24b9c17b824faba8f6193822e5dcc41e7ddedd4abb0dc1c4811a15dd
  Stored in directory: /root/.cache/pip/wheels/ed/2c/5f/a0ccc5955d44d2cea78729f4425e73f818d2629517f7af0f8b
Successfully built summa


In [ ]:
from summa.summarizer import summarize

summarized_df = []
for i in df['Body']:
  summarized_df.append(summarize(str(i), ratio=0.2))
df.insert(4,'summary', summarized_df,True)
df.head(5)

,index,Title,Time,Url,summary,Body
0,0,Sparta continues to feel tremors after 5.1 mag...,2020-08-11 12:17 UTC,https://myfox8.com/news/sparta-continues-to-fe...,FOX8 WGHP Please enter a search term.\nPlease ...,FOX8 WGHP Please enter a search term. Please e...
1,1,Sparta hit by 2.2 magnitude tremor morning aft...,2020-08-10 12:55 UTC,https://myfox8.com/news/sparta-hit-by-2-2-magn...,FOX8 WGHP Please enter a search term.\nPlease ...,FOX8 WGHP Please enter a search term. Please e...
2,2,"6.5 Earthquake in Idaho: Map, Videos & Distanc...",2020-04-01 00:30 UTC,https://heavy.com/news/2020/03/earthquake-idaho/,,\n\t\t\tGlenn Douglas Perry was named as th...
3,3,"Powerful earthquake shakes southern Mexico, at...",2020-06-23 19:46 UTC,https://www.8newsnow.com/news/local-news/power...,KLAS Please enter a search term.\nPlease enter...,KLAS Please enter a search term. Please enter ...
4,4,"WATCH: 'As If Things Couldn't Get Worse,' Garc...",2020-06-04 02:47 UTC,https://heavy.com/news/2020/06/as-if-things-co...,#earthquake pic.twitter.com/xGgVEzzRO4 — Niall...,\n\n\t\t\t\t\t\t\tTwitter/MayorofLA\t\t\t\t\t\...


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 2.2 Location Extraction - location tagging

In [ ]:
for i in df:
    body = df['summary'] + df['Title']
    cities = []
    for j in range(len(body)):
      place_entity = locationtagger.find_locations(text=str(body[j]))
      city = place_entity.cities
      if len(city) == 0:
        citi = ('')
      else:
        citi = city[0]
      region = place_entity.regions
      cities.append(citi)
    df['Location'] = cities

In [ ]:
coordinate_info = pd.read_csv(gdrive_path + 'uscities_coordinate.csv')
coord = coordinate_info[['city','lat','lng']]
coord_city = coord['city'].tolist()
coord_lat = coord['lat'].tolist()
coord_lon = coord['lng'].tolist()
matching_cities =[]
matching_index = []
latitudes = []
longitudes = []
for element in cities:
  if element in coord_city:
    res = element
  elif element not in coord_city:
    res = []
  matching_cities.append(res)

for element in matching_cities:
  if len(element) > 0:
    ind = coord_city.index(element)
    matching_index.append(ind)
  else:
    ind = []
    matching_index.append(ind)

for element in matching_index:
  if element !=[]:
    latitude = coord_lat[element]
    longitude = coord_lon[element]
    latitudes.append(latitude)
    longitudes.append(longitude)
  else:
    ind= ''
    latitudes.append(ind)
    longitudes.append(ind)

In [ ]:
df.dropna().reset_index().to_csv(gdrive_path+'news_all_info.csv',index=False)

In [ ]:
info_df = pd.read_csv(gdrive_path+"news_all_info.csv").dropna().reset_index()

In [ ]:
info_df.to_csv(gdrive_path+'news_all_info.csv',index=False)

### 2.3 Split dataset into different seasons (Spring, Summer, Fall, Winter)

In [ ]:
def seasonal_split(df):
    
    sp_df = pd.DataFrame(); su_df = pd.DataFrame(); fa_df = pd.DataFrame(); wn_df = pd.DataFrame()
    
    for i in range(len(df)):
        
        month = int(df['Time'][i].split("-")[1])
        row = pd.DataFrame(df.iloc[i]).T
        
        if month in range(3, 6):
            sp_df = pd.concat([sp_df, row], ignore_index=True)
        elif month in range(6, 9):
            su_df = pd.concat([su_df, row], ignore_index=True)
        elif month in range(9, 12):
            fa_df = pd.concat([fa_df, row], ignore_index=True)
        elif month < 3 or month >= 11:
            # wn_df.append(df.iloc[i])
            wn_df = pd.concat([wn_df, row], ignore_index=True)
            
    return sp_df, su_df, fa_df, wn_df

In [ ]:
sp_df, su_df, fa_df, wn_df = seasonal_split(info_df)

In [ ]:
sp_df.to_csv(gdrive_path + 'spring2020_eq.csv', index=False)
su_df.to_csv(gdrive_path + 'summer2020_eq.csv', index=False)
fa_df.to_csv(gdrive_path + 'fall2020_eq.csv', index=False)
wn_df.to_csv(gdrive_path + 'winter2020_eq.csv', index=False)

In [ ]:
sp_df.shape, su_df.shape, fa_df.shape, wn_df.shape

((179, 9), (253, 9), (192, 9), (286, 9))

## 2.4 Classification model training
Label: 
0 if the article main text is not relevant to earthquake
1 if relevant so the right data is extracted

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification
import torch
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
from tqdm import tqdm
import os
import evaluate
from sklearn.dummy import DummyClassifier
#from datasets import load_dataset
from collections import Counter, defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Verify there's at least one GPU sitting around (or not if you want to try CPU!)
torch.cuda.device_count()

1

In [ ]:
fall_full = pd.read_csv(gdrive_path + "annotated_fall2020_eq.csv", encoding = "windows-1252").dropna()

In [ ]:
train_df, eval_df, test_df = np.split(fall_full.sample(frac=1, random_state=42), [int(.6*len(fall_full)), int(.8*len(fall_full))])

In [ ]:
# train_df, eval_df = train_test_split(fall_full, test_size = 0.33, random_state=42)

# train_df = train_df.reset_index()
# eval_df = eval_df.reset_index()
# test_df = test_df.reset_index()

# train_df["label"] = train_df["label"].astype(int)
# eval_df["label"] = eval_df["label"].astype(int)
# test_df["label"] = test_df["label"].astype(int)
test_df

,index,Title,Time,Url,label,Body,summary,Location,latitude,longitude
168,2098,Magnitude 3.6 earthquake felt across Massachus...,2020-11-08 16:17 UTC,https://www.pennlive.com/nation-world/2020/11/...,1.0,A magnitude 3.6 earthquake was felt across Mas...,A magnitude 3.6 earthquake was felt across Mas...,Springfield,42.1155,-72.5395
135,1649,Earthquake strums Bruce Springsteen's New Jers...,2020-09-09 13:22 UTC,https://www.inquirer.com/news/earthquake-new-j...,1.0,A mild earthquake shook the area around Freeho...,"?€?It sounded like an explosion nearby, follow...",Sanders,35.2095,-109.3224
160,1961,3.1 magnitude earthquake shakes part of New Je...,2020-09-09 10:53 UTC,https://www.pennlive.com/nation-world/2020/09/...,1.0,A U.S. Geological Survey map showing the locat...,A U.S. Geological Survey map showing the locat...,Virginia,47.5168,-92.5127
176,2223,4.5 earthquake north of Willow shakes Southcen...,2020-11-26 21:58 UTC,https://www.adn.com/alaska-news/mat-su/2020/11...,1.0,A magnitude 4.5 earthquake north of Willow ra...,That 3.4 quake occurred at a depth of 8.6 mile...,Miles,31.6006,-100.1853
63,341,"3.1 magnitude earthquake hits New Jersey, USGS...",2020-09-09 14:31 UTC,https://www.lex18.com/news/national/3-1-magnit...,1.0,"Menu EAST FREEHOLD, N.J. ?€? A 3.1 magnitude e...",?€? A 3.1 magnitude earthquake was detected We...,Milford,38.9091,-75.4227
54,327,Tsunami warning issued after 7.5-magnitude ear...,2020-10-19 23:04 UTC,https://www.wxii12.com/article/tsunami-warning...,1.0,"The latest breaking updates, delivered straigh...","The magnitude 7.5 earthquake near Sand Point, ...",Canadian,35.9096,-100.3838
107,756,Weak earthquake occurs on Alabama-Florida line...,2020-09-03 17:02 UTC,https://www.abcactionnews.com/news/state/weak-...,0.0,"Menu BREWTON, Ala. (AP) ?€? No damage is being...",(AP) ?€? No damage is being reported after a w...,Florida,18.3643,-66.5611
50,323,7.5-magnitude earthquake near Alaska generates...,2020-10-19 23:57 UTC,https://www.wmur.com/article/tsunami-warning-i...,1.0,"The latest breaking updates, delivered straigh...","The magnitude 7.5 earthquake near Sand Point, ...",Canadian,35.9096,-100.3838
188,2242,Powerful earthquake near Alaska triggers tsuna...,2020-10-19 21:23 UTC,https://www.foxla.com/news/powerful-earthquake...,1.0,Share This map shows the epicenter of an earth...,"""SOUTH ALASKA AND THE ALASKA PENINSULA, Pacifi...",Pacific,47.2610,-122.2507
58,331,Alaska earthquake measuring 7.4 triggers tsuna...,2020-10-19 21:59 UTC,https://abc13.com/weather/alaska-earthquake-me...,1.0,WATCH LIVE ABC7 has compiled a list of items t...,WATCH LIVE ABC7 has compiled a list of items t...,Juneau,58.4546,-134.1739


In [ ]:
# Tokenize train and evaluation datasets with BertTokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(data):
    return tokenizer(data, padding=True, truncation=True, max_length=256)

tokenized_X_train = preprocess_function(list(train_df['summary']))
tokenized_X_eval = preprocess_function(list(eval_df['summary']))
tokenized_X_test = preprocess_function(list(test_df['summary']))

In [ ]:
# Define a Class to prepare the dataset in desired format to feed in Trainer class
class DataProcess(torch.utils.data.Dataset):
  def __init__(self, encodings, labels = None):
    self.encodings = encodings
    self.labels = labels
  
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item["labels"] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

In [ ]:
train_dataset = DataProcess(tokenized_X_train, list(train_df['label']))
eval_dataset = DataProcess(tokenized_X_eval, list(eval_df['label']))
test_dataset = DataProcess(tokenized_X_test, list(test_df['label']))

In [ ]:
device = torch.device("cuda")

In [ ]:
from sklearn.metrics import mean_squared_error

def compute_metrics(pred):
  scores = pred.label_ids
  prediction = pred.predictions.squeeze()
  return {"rmse": np.sqrt(mean_squared_error(scores, prediction))}

# Huggingface will generate model checkpoints and these can 
# take up a lot of space, so be sure to set this directory somewhere where
# you can check

# This code block is used so we can have a deterministic training procedure
# (i.e., get the same result each time)
# model_name = "microsoft/MiniLM-L12-H384-uncased"
output_dir = '.'

def model_init():
    from transformers.models.bert.modeling_bert import BertForSequenceClassification
    bertmodel = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1) # label of search word relevancy score
    bertmodel = bertmodel.to(device)

    return bertmodel #AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
     
training_rel = TrainingArguments(
    output_dir=output_dir,
    #evaluation_strategy="step",
    #eval_steps = 500,
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    #weight_decay=0.01,
    do_eval=True,
    seed=1234,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_dir=output_dir + 'logs/',
    load_best_model_at_end=True,
    metric_for_best_model="rmse",
    #max_steps = 1000
)

In [ ]:
model_init()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
device = torch.device("cuda")

In [ ]:
trainer = Trainer(
    model_init=model_init,    
    args=training_rel,
    train_dataset = train_dataset, #pd.DataFrame(X_train_sparse), #{'input_ids': data_dict["X_train"].toarray(), 'labels': y_train}, #data_dict["X_train"].todense(),  #(X_train_sparse).todense(), 
    eval_dataset = eval_dataset, #pd.DataFrame(X_val_sparse), #{'input_ids': data_dict["X_val"].toarray(), 'labels': y_val}, # #data_dict["X_val"].todense(), #(X_val_sparse).todense(),    
    compute_metrics = compute_metrics,
    tokenizer=tokenizer,
    #data_collator=data_collator,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Rmse
1,No log,2.657724,1.630253
2,No log,2.703507,1.644235
3,No log,2.748174,1.657762
4,No log,2.656942,1.630013
5,No log,2.679505,1.636919
6,No log,2.695216,1.641711
7,No log,2.679118,1.636801
8,No log,2.681074,1.637398


TrainOutput(global_step=64, training_loss=0.0720447227358818, metrics={'train_runtime': 86.0748, 'train_samples_per_second': 10.595, 'train_steps_per_second': 0.744, 'total_flos': 119977564004352.0, 'train_loss': 0.0720447227358818, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.7481741905212402,
 'eval_rmse': 1.6577619314193726,
 'eval_runtime': 0.6329,
 'eval_samples_per_second': 60.043,
 'eval_steps_per_second': 4.74,
 'epoch': 8.0}

In [ ]:
predictions = trainer.predict(test_dataset)[0]
predictions

array([[0.8420019 ],
       [0.9899676 ],
       [0.94107604],
       [0.9240194 ],
       [0.88997495],
       [0.9112953 ],
       [0.9760797 ],
       [0.9112953 ],
       [0.93476725],
       [0.93409383],
       [0.95576954],
       [0.9112953 ],
       [0.9083522 ],
       [0.93409383],
       [0.7669759 ],
       [0.92109716],
       [0.8829567 ],
       [0.97072244],
       [0.8041403 ],
       [0.92109716],
       [0.604928  ],
       [0.9261601 ],
       [0.78308624],
       [1.0047127 ],
       [1.0237192 ],
       [0.6541977 ],
       [0.88997495],
       [0.8652497 ],
       [0.9112953 ],
       [1.000205  ],
       [1.0197719 ],
       [0.8961951 ],
       [0.9083523 ],
       [0.8519683 ],
       [1.0047128 ],
       [0.9083523 ],
       [0.946926  ],
       [0.9210971 ],
       [0.96882933]], dtype=float32)

In [ ]:
len(predictions)

39

In [ ]:
pred_arr = np.array([predictions[i].item() for i in range(len(predictions))])
pred_arr

array([0.8420019 , 0.9899676 , 0.94107604, 0.9240194 , 0.88997495,
       0.9112953 , 0.9760797 , 0.9112953 , 0.93476725, 0.93409383,
       0.95576954, 0.9112953 , 0.9083522 , 0.93409383, 0.7669759 ,
       0.92109716, 0.8829567 , 0.97072244, 0.8041403 , 0.92109716,
       0.604928  , 0.9261601 , 0.78308624, 1.0047127 , 1.0237192 ,
       0.6541977 , 0.88997495, 0.8652497 , 0.9112953 , 1.000205  ,
       1.0197719 , 0.8961951 , 0.9083523 , 0.8519683 , 1.0047128 ,
       0.9083523 , 0.946926  , 0.9210971 , 0.96882933])

In [ ]:
np.array(test_df["label"])

array([1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

#test_df["label"] = test_df["label"].astype(float)
classification_report(np.array(test_df["label"]), pred_arr.round())

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n         0.0       0.00      0.00      0.00         3\n         1.0       0.92      1.00      0.96        36\n\n    accuracy                           0.92        39\n   macro avg       0.46      0.50      0.48        39\nweighted avg       0.85      0.92      0.89        39\n'

# 3. Geospatial Map Visualization

In [ ]:
spring20_df = pd.read_csv(gdrive_path + "spring2020_eq.csv")
summer20_df = pd.read_csv(gdrive_path + "summer2020_eq.csv")
fall20_df = pd.read_csv(gdrive_path + "fall2020_eq.csv")
winter20_df = pd.read_csv(gdrive_path + "summer2020_eq.csv")

In [ ]:
def folium_map(d1, d2, d3, d4, colors = ['green', 'red', 'lightblue', 'gray'], markers = ['+', 'x', '', '.']):
  import folium
  import math
  map = folium.Map(width=1400, height=750, location=(45,-93),zoom_start=4.5, control_scale=True)

  for data, col, mark in  zip([d1, d2, d3, d4], colors, markers):
    for i in range(len(data)):
      if data.latitude[i] != '':
        folium.Marker(location=[data.latitude[i], data.longitude[i]],icon=folium.Icon(color=col, icon=mark)).add_to(map)
      else:
        pass

  f1=folium.FeatureGroup("Natural Disaster")
  f1.add_to(map)
  
  return map

In [ ]:
folium_map(spring20_df, summer20_df, fall20_df, winter20_df)

# Things I have tried but could not make work 
### 1. Tried but failed - Compute Spatial Correlation 
### 2. Yet to try - Keyword Sentitivity (possibly search word or certain word in the text)
### 3. Yet to try - Relationship between Magnitude, Location, and Season

In [ ]:
# pip install haversine

In [ ]:
import pandas as pd
import numpy as np
import haversine as hs
import folium 

In [ ]:
# concatenating lat and long to create a consolidated location as accepted by havesine function
spring_loc['coor'] = list(zip(spring_loc.latitude, spring_loc.longitude))
summer_loc['coor'] = list(zip(summer_loc.latitude, summer_loc.longitude))

In [ ]:
spring_loc

,index,Title,Time,Url,Body,summ,Location,latitude,longitude,coor
0,98,3.5-magnitude earthquake recorded near Anza,2020-04-05 22:59 UTC,https://fox5sandiego.com/news/earthquakes/3-5-...,FOX 5 San Diego Please enter a search term. Pl...,— A 3.5-magnitude earthquake struck near Anza ...,Anza,33.5678,-116.6966,"(33.5678, -116.6966)"
1,99,[BREAKING] 4.9 Magnitude Earthquake Shakes Sou...,2020-04-04 03:21 UTC,https://www.techtimes.com/articles/248585/2020...,"As explained, an average of 234 earthquakes ...","As explained, an average of 234 earthquakes wi...",California,38.2969,-76.4949,"(38.2969, -76.4949)"
2,100,6.5-Magnitude Earthquake Shakes Idaho,2020-04-01 11:04 UTC,https://www.newsmax.com/newsfront/earthquake-i...,"\n\nWednesday, 01 April 2020 07:04 AM EDT\n\n ...",The earthquake added stress during an already ...,California,38.2969,-76.4949,"(38.2969, -76.4949)"
3,107,Petrolia earthquake: 5.2 magnitude quake strik...,2020-03-18 22:52 UTC,https://abc7news.com/weather/petrolia-earthqua...,WATCH LIVE If you haven't put together an eart...,(KGO) -- A large earthquake struck off the coa...,Petrolia,34.0131,-98.2313,"(34.0131, -98.2313)"
4,167,Deadly Earthquake Strikes Southwest China,2020-05-19 05:26 UTC,https://www.voanews.com/east-asia-pacific/dead...,\n\n\nPrint\n\n At least four people are dead ...,At least four people are dead and 23 others in...,China,30.0522,-94.3331,"(30.0522, -94.3331)"
...,...,...,...,...,...,...,...,...,...,...
174,2168,Utah earthquake of 5.7-magnitude rocks Salt La...,2020-03-18 16:04 UTC,https://www.foxnews.com/us/utah-earthquake-5-7...,"\n This material may not be published, br...","This material may not be published, broadcast,...",Salt Lake City,40.7776,-111.9311,"(40.7776, -111.9311)"
175,2173,5.6 magnitude earthquake shakes northern Mexico,2020-03-07 04:32 UTC,https://kvoa.com/news/2020/03/06/5-5-magnitude...,The two nearest cities to the earthquake are ...,"Lydia Isenberger, who lives west of A Mountain...",Mexico,39.1625,-91.8712,"(39.1625, -91.8712)"
176,2174,UPDATE: 3.4 magnitude earthquake hits near Ale...,2020-03-06 02:08 UTC,https://kfor.com/news/local/update-3-4-magnitu...,KFOR.com Oklahoma City Please enter a search t...,KFOR.com Oklahoma City Please enter a search t...,Oklahoma City,35.4676,-97.5136,"(35.4676, -97.5136)"
177,2295,6.4-magnitude earthquake in western Nevada fel...,2020-05-15 13:21 UTC,https://fox5sandiego.com/news/earthquakes/6-4-...,FOX 5 San Diego Please enter a search term. Pl...,FOX 5 San Diego Please enter a search term.\nP...,Nevada,37.8445,-94.3492,"(37.8445, -94.3492)"


In [ ]:
import pandas as pd
from geopy.distance import great_circle

# calculate the distance between each pair of points using great_circle
distances = []
for idx1, row1 in df1.iterrows():
    for idx2, row2 in df2.iterrows():
        distance = great_circle((row1['latitude'], row1['longitude']), 
                                (row2['latitude'], row2['longitude'])).km
        distances.append(distance)

# calculate the correlation between the two datasets
correlation = pd.Series(distances).corr()
print('Spatial correlation:', correlation)


In [ ]:
import folium

# create a map
m = folium.Map(location=[df1['latitude'].mean(), df1['longitude'].mean()], zoom_start=4.5)

# add markers for the points in df1 and df2
for idx, row in df1.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup='df1').add_to(m)
for idx, row in df2.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup='df2').add_to(m)

# display the map

In [ ]:
def create_map(data):
    m = folium.Map(location=[data.Lat.mean(), data.Lon.mean()], zoom_start=5)
    
    for _, row in data.iterrows():
         
        if row['Condition']=='Out':
            cluster_colour='blue'
        else:
            cluster_colour = 'green'
        
        folium.CircleMarker(
            location= [row['Lat'],row['Lon']],
            radius=5,
            popup= row[['Condition','Location']],
            color=cluster_colour,
            fill=True,
            fill_color=cluster_colour
            ).add_to(m)
        
    return m